---

<p style="text-align: center; font-size: 25px;">
    <strong>
          Final attemt to collect data manually
    </strong>
</p>

---

In [ ]:
import pandas as pd
import os

def add_to_0csv(goods_url, reviews_url, filename=f'database/0.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'0-{len(df):06}')
    
    new_row = {'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added.")
        return
    
    df = df.append(new_row, ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully.")

In [23]:
def get_reviews_url_by_index(index, filename):

    df = pd.read_csv(filename)
    
    index = index.split('-')[1]
    index = int(index)
    
    if index < 0 or index >= len(df):
        raise IndexError("ID out of range of DataFrame")
    
    reviews_url = df.loc[index, 'reviews_url']
    
    return reviews_url

'''index = 0
try:
    reviews_url = get_reviews_url_by_index(index)
    print(f"The reviews_url at index {index} is: {reviews_url}")
except IndexError as e:
    print(e)'''


'index = 0\ntry:\n    reviews_url = get_reviews_url_by_index(index)\n    print(f"The reviews_url at index {index} is: {reviews_url}")\nexcept IndexError as e:\n    print(e)'

In [54]:
add_to_0csv('https://www.amazon.es/Russell-Hobbs-24031-56-temporizador-programable/dp/B07G8JQSQR/ref=sr_1_7?dib=eyJ2IjoiMSJ9.IrJ3PxC-nG-YqhUNnIX9Jiw48ZDvNl0MF5wJiRGSjvyBvn3Ea4STk7gQzz_U5juaF9Gd55ePZJRnt0cywE-JQBEwH4ig2xjXmYyJSVhv2KA8ikTXZfRh9vbdueRv01Xn5qkyvx_v9oTfdOdDoBoO8XzsgPSvpSdsIUpr9VvdKMv3Xj9pVrLvBSjil-j0PGkVrjLpzplF1lhQ482pXDhTyruERgk36NxP9kKM6hiUtP94kI5vofPPR4WBmnknyDFBGbV3R1_N-NndQ9hIRv_4nSfZydEJgqpatQBOwEgcqiA.O0Qc8bGdd4ab0WFE_W89ciuftYartK44g_8E5n5G2lE&dib_tag=se&qid=1721654564&s=kitchen&sr=1-7', 'https://www.amazon.es/Russell-Hobbs-24031-56-temporizador-programable/product-reviews/B07G8JQSQR/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

In [106]:
add_to_0csv('https://www.amazon.es/Jocca-Cafetera-Italiana-El%C3%A9ctrica', 'https://www.amazon.es/Jocca-Cafetera-Italiana-El%C3%A9ctrica-autom%C3%A1tico/product-reviews/B0CS3MCF9J/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

Row https://www.amazon.es/Jocca-Cafetera-Italiana-El%C3%A9ctrica added successfully.


In [307]:
add_to_0csv('https://www.amazon.es/dp/B08H585PJ2/ref=sspa_dk_detail_1?pd_rd_i=B08H585PJ2&pd_rd_w=bH8km&content-id=amzn1.sym.fa290e14-a761-4185-b21b-f2448830508b&pf_rd_p=fa290e14-a761-4185-b21b-f2448830508b&pf_rd_r=HAJNY0ATPX2NBZCXJQ14&pd_rd_wg=GzCC8&pd_rd_r=20bc4717-0293-4421-b315-9582bba3493f&sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWxfdGhlbWF0aWM&th=1', 'https://www.amazon.es/Easyworkz-Eclipse-filtraci%C3%B3n-resistente-borosilicato/product-reviews/B08H585PJ2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

Duplicate row found. The row https://www.amazon.es/dp/B08H585PJ2/ref=sspa_dk_detail_1?pd_rd_i=B08H585PJ2&pd_rd_w=bH8km&content-id=amzn1.sym.fa290e14-a761-4185-b21b-f2448830508b&pf_rd_p=fa290e14-a761-4185-b21b-f2448830508b&pf_rd_r=HAJNY0ATPX2NBZCXJQ14&pd_rd_wg=GzCC8&pd_rd_r=20bc4717-0293-4421-b315-9582bba3493f&sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWxfdGhlbWF0aWM&th=1 will not be added.


In [510]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random
from tqdm import tqdm

def get_urls(ID, filename):
    
    try:
        base_url = get_reviews_url_by_index(ID, filename)
        print(f"The reviews_url at index {ID} is: {base_url}")
    except IndexError as e:
        print(e)
    
    filters = [
        'sortBy=recent',
        'sortBy=helpful',
        'sortBy=rating',
        'filterByStar=one_star',
        'filterByStar=two_star',
        'filterByStar=three_star',
        'filterByStar=four_star',
        'filterByStar=five_star'
    ]
    
    list_urls = []
    for filter_option in filters:
        for page in range(1, 11):
            
            #updating the filter and page number 
            updated_url = f"{base_url}&{filter_option}&pageNumber={page}"
            list_urls.append(updated_url)
    
    return list_urls

def scrape_reviews(urls):
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    DRIVER_PATH = '/Users/apple/Downloads/Project_Gnomi_Huekradi/chromedriver'

    service = Service(DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    
    reviews = []
    
    for url in tqdm(urls, desc='Processing URLs', unit='URL'): 
        driver.get(url)

        time.sleep(1)

        # Getting reviews
        review_blocks = driver.find_elements(By.CSS_SELECTOR, '.a-section.review')
        for review_block in review_blocks:
            title = review_block.find_element(By.CSS_SELECTOR, '.review-title-content').text.strip()
            rating = review_block.find_element(By.CSS_SELECTOR, '.a-icon-alt').get_attribute('textContent').strip()
            body = review_block.find_element(By.CSS_SELECTOR, '[data-hook="review-body"]').text.strip()
            author = review_block.find_element(By.CSS_SELECTOR, '.a-profile-name').text.strip()
            date = review_block.find_element(By.CSS_SELECTOR, '.review-date').text.strip()

            reviews.append({
                'title': title,
                'rating': rating,
                'body': body,
                'author': author,
                'date': date,
            })
    
    print("Parsing is done.")
    driver.quit()
    return reviews

# Removing duplicates and saving data
def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.drop_duplicates(subset=['title', 'body', 'author'], inplace=True)
    df.to_csv(filename, index=False, encoding='utf-8')
    
    return int(df.shape[0])

if __name__ == "__main__":
    
    start_time = time.perf_counter() 
    
    #queue = [53, 54, 55, 56] # from 233 to 276
    
    queue = [278]
    
    #up to page 45
    
    for i in queue:
        
        ID = f'0-{i:06}'
    
        urls = get_urls(ID, f'database/0.csv')
        reviews = scrape_reviews(urls)
    
        adress = f'database/reviews/0-{i:06}.csv'
        l = save_to_csv(reviews, adress)
        print(f'Saved {l} individual reviews in database/reviews/0-{ID}.csv\n------------------------------------------------------------------')
    
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f'Time taken: {elapsed_time:.2f} seconds')

The reviews_url at index 0-000278 is: https://www.amazon.es/Melitta-Cafetera-Filtro-Vidrio-Capacidad/product-reviews/B07GFPJ6YY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


Processing URLs: 100%|██████████| 80/80 [02:41<00:00,  2.02s/URL]


Parsing is done.
Saved 509 individual reviews in database/reviews/0-0-000278.csv
------------------------------------------------------------------
Time taken: 162.63 seconds


In [515]:
testing_dataframes = [] 
for j in range(279):
    
    
    file_path = os.path.join('', f'database/reviews/0-{j:06}.csv')
    # Чтение CSV файла и добавление его в список
    try:
        a = pd.read_csv(file_path)
        testing_dataframes.append(a)

    except:
        print(f'No data in database/reviews/0-{j:06}.csv')
# Объединение всех DataFrame в один

#a = pd.read_csv('normal/reviews_filtered.csv')
#testing_dataframes.append(a)

testing_combined_df = pd.concat(testing_dataframes, ignore_index=True)
testing_combined_df.drop_duplicates(inplace=True)

In [516]:
testing_combined_df.shape[0]

54660

In [9]:
import spacy
from textblob import TextBlob

nlp = spacy.load('en_core_web_sm')

def analyze_sentiment(review):

    doc = nlp(review) # tokenizer 
    
    processed_review = " ".join([token.text for token in doc]) # adjusting for TextBlob

    blob = TextBlob(processed_review)
    
    polarity = blob.sentiment.polarity # getting polarity between -1.0 and 1.0
    return polarity

reviews = [
    "I absolutely love this product! It has changed my life for the better.",
    "This is the worst purchase I have ever made. Completely disappointed.",
    "The product is okay, not great but not terrible either.",
    "Amazing quality! Exceeded my expectations.",
    "Terrible service. Will not buy again."
]

for review in reviews:
    polarity = analyze_sentiment(review)
    sentiment = "Positive" if polarity > 0 else "Negative" if polarity < 0 else "Neutral"
    print(f"Review: {review}\nSentiment: {sentiment} (Polarity: {polarity})\n")


Review: I absolutely love this product! It has changed my life for the better.
Sentiment: Positive (Polarity: 0.5625)

Review: This is the worst purchase I have ever made. Completely disappointed.
Sentiment: Negative (Polarity: -0.875)

Review: The product is okay, not great but not terrible either.
Sentiment: Positive (Polarity: 0.19999999999999998)

Review: Amazing quality! Exceeded my expectations.
Sentiment: Positive (Polarity: 0.7500000000000001)

Review: Terrible service. Will not buy again.
Sentiment: Negative (Polarity: -1.0)



In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random
from tqdm import tqdm

def add_to_2(goods_url, reviews_url, filename=f'database/2.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['ID', 'goods_url', 'reviews_url'])
    
    next_id = os.path.join('', f'2-{len(df):06}')
    
    new_row = pd.DataFrame([{'ID': next_id, 'goods_url': goods_url, 'reviews_url': reviews_url}])
    
    if not df[(df['goods_url'] == goods_url) & (df['reviews_url'] == reviews_url)].empty:
        print(f"Duplicate row found. The row {goods_url} will not be added to {filename}.")
        return
    
    df = pd.concat([df, new_row], ignore_index=True)
    
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Row {goods_url} added successfully to {filename}")

In [ ]:
add_to_2()